In [1]:
%load_ext autoreload
%autoreload 2

# データ収集用Jupyterノートブック


In [2]:
# データ収集用Jupyterノートブック

import sys
import os
import json
import datetime as dt
import pandas as pd
import asyncio

# データコレクターモジュールをインポート
from data_collector import BTCDataCollector

# 設定ファイルから読み込む
def load_config(config_path="config/data_config.json"):
    try:
        with open(config_path, "r") as f:
            config = json.load(f)

        # 日付文字列をdatetimeオブジェクトに変換
        if "start_date" in config and config["start_date"]:
            config["start_date"] = dt.datetime.fromisoformat(config["start_date"])
        if "end_date" in config and config["end_date"]:
            config["end_date"] = dt.datetime.fromisoformat(config["end_date"])
        else:
            config["end_date"] = dt.datetime.now()

        return config
    except Exception as e:
        print(f"設定ファイルの読み込みエラー: {e}")
        return None

# 設定ファイルから読み込み
config = load_config()

# または直接設定
if config is None:
    config = {
        "exchange": "bybit",
        "symbol": "BTCUSDT",
        "timeframe": "5",
        "start_date": dt.datetime(2023, 1, 1),
        "end_date": dt.datetime.now(),
        "output_dir": "data/raw",
        "output_filename": "btcusd_5m_data.csv",
        "api_keys": {
            "bybit": ["YOUR_API_KEY", "YOUR_API_SECRET"]  # 実際のキーに置き換え
        },
        "use_direct_api": True  # True: 直接APIを使用, False: pybottersを使用
    }

# API キーを設定（必要な場合）
# config["api_keys"]["bybit"] = ["YOUR_ACTUAL_API_KEY", "YOUR_ACTUAL_API_SECRET"]

# 設定内容を表示（APIキーは隠す）
print_config = config.copy()
print_config["api_keys"] = {k: ["***", "***"] for k, v in config["api_keys"].items()}
print("使用する設定:")
for key, value in print_config.items():
    print(f"  {key}: {value}")

# データ収集と保存の関数
async def collect_and_save_data(config):
    """データを収集して保存する関数"""
    collector = BTCDataCollector(config)
    df = await collector.collect_historical_data()
    summary = collector.save_data(df)

    # サマリー出力
    print("\n=== データ収集結果サマリー ===")
    print(f"状態: {summary['status']}")
    print(f"行数: {summary['rows']}")
    print(f"期間: {summary['start_date']} から {summary['end_date']}")
    if summary['status'] == 'success':
        print(f"価格統計（終値）: 最小={summary['data_stats']['close']['min']:.2f}, "
              f"最大={summary['data_stats']['close']['max']:.2f}, "
              f"平均={summary['data_stats']['close']['mean']:.2f}")
        print(f"出来高合計: {summary['data_stats']['volume']['total']:.2f}")
        print(f"欠損値: {sum(summary['missing_values'].values())}")

    return df, summary

# データ収集実行
df, summary = await collect_and_save_data(config)

# データ確認
if 'df' in locals() and not df.empty:
    print("データサンプル（先頭5行）:")
    print(df.head())

    print("\n基本統計:")
    print(df.describe())

    print(f"\n合計レコード数: {len(df)}")
else:
    print("データの収集に失敗したか、データが空です。")

2025-04-26 13:10:26,048 - bybit_collector - INFO - BTCUSDTの5分足データを2023-01-01 00:00:00から2025-04-26 13:10:26.047656まで収集します
2025-04-26 13:10:26,048 - bybit_collector - INFO - データ取得を847個のチャンクに分割しました


使用する設定:
  exchange: bybit
  symbol: BTCUSDT
  timeframe: 5
  start_date: 2023-01-01 00:00:00
  end_date: 2025-04-26 13:10:26.047656
  output_dir: data/raw
  output_filename: btcusd_5m_data.csv
  api_keys: {'bybit': ['***', '***']}
  use_direct_api: True


データ取得:   0%|          | 0/847 [00:00<?, ?it/s]

2025-04-26 13:10:41,080 - bybit_collector - INFO - データ収集完了。合計 169358 行のデータを取得
2025-04-26 13:10:41,686 - bybit_collector - INFO - データを data\raw\btcusd_5m_data.csv に保存しました。データサイズ: (169358, 6)



=== データ収集結果サマリー ===
状態: success
行数: 169358
期間: 2022-12-31T22:25:00 から 2025-04-26T04:10:00
価格統計（終値）: 最小=16508.00, 最大=109059.40, 平均=53393.19
出来高合計: 65765932.16
欠損値: 0
データサンプル（先頭5行）:
                        open     high      low    close   volume      turnover
timestamp                                                                     
2022-12-31 22:25:00  16545.0  16568.5  16544.5  16557.0  199.789  3.308472e+06
2022-12-31 22:30:00  16557.0  16569.0  16557.0  16563.0  265.757  4.402056e+06
2022-12-31 22:35:00  16563.0  16563.0  16544.5  16546.0  165.466  2.738463e+06
2022-12-31 22:40:00  16546.0  16552.0  16528.5  16551.5  295.414  4.885751e+06
2022-12-31 22:45:00  16551.5  16552.0  16533.0  16534.0   78.281  1.295245e+06

基本統計:
                open           high            low          close  \
count  169358.000000  169358.000000  169358.000000  169358.000000   
mean    53392.910851   53441.023725   53344.119182   53393.188712   
std     25365.104302   25392.996380   25337.123430  

# 特徴量エンジニアリング用Jupyter Notebook


In [3]:
# 特徴量エンジニアリング用Jupyter Notebook

import sys
import os
import json
import pandas as pd
import numpy as np
from IPython.display import display

# data_processorモジュールをインポート
sys.path.append(os.path.abspath(os.getcwd()))
from data_processor import generate_features

# 特徴量生成の設定（必要に応じてカスタマイズ）
config = {
    "input_dir": "data/raw",
    "input_filename": "btcusd_5m_data.csv",
    "output_dir": "data/processed",
    "output_filename": "btcusd_5m_features.csv",
    "features": {
        "price_change": True,
        "volume_change": True,
        "moving_averages": True,
        "rsi": True,
        "high_low_distance": True,
        "bollinger_bands": True,
        "macd": True,
        "stochastic": True
    },
    "ma_periods": [5, 10, 20, 50, 100, 200],
    "rsi_period": 14,
    "bollinger_period": 20,
    "bollinger_std": 2,
    "macd_params": {"fast": 12, "slow": 26, "signal": 9},
    "stochastic_params": {"k": 14, "d": 3, "slowing": 3},
    "target_periods": [1, 2, 3],
    "classification_threshold": 0.0005
}

# 特徴量生成の実行
feature_df, report = generate_features(config)

# 結果の表示
print("\n=== 特徴量生成結果 ===")
print(f"状態: {report['status']}")
print(f"行数: {report['rows']}")
print(f"列数: {report['columns']}")
print(f"期間: {report['start_date']} から {report['end_date']}")
print(f"欠損値: {report['missing_values']}")

print("\n特徴量グループごとの数:")
for group, count in report['feature_counts'].items():
    print(f"  {group}: {count}")

print("\n目標変数の分布:")
for target, distribution in report['target_distribution'].items():
    print(f"  {target}: {distribution}")

# 主要特徴量の基本統計量を表示
main_features = [
    'close', 'volume',
    'price_change_pct_1', 'volume_change_pct_1',
    'rsi', 'bb_width', 'macd', 'stoch_k',
    'target_price_change_pct_1'
]

if all(feat in feature_df.columns for feat in main_features):
    print("\n主要特徴量の基本統計量:")
    stats = feature_df[main_features].describe()
    for col in stats.columns:
        print(f"\n{col}:")
        for stat, value in stats[col].items():
            print(f"  {stat}: {value:.6f}")

# 相関係数の表示（主要な特徴量のみ）
if all(feat in feature_df.columns for feat in main_features):
    main_features_with_target = main_features + ['target_price_direction_1']
    print("\n主要特徴量と目標変数(1期先)の相関係数:")
    target_corr = feature_df[main_features_with_target].corr()['target_price_direction_1'].drop('target_price_direction_1')
    for feat, corr in target_corr.items():
        print(f"  {feat}: {corr:.6f}")

# 特徴量の頻度カウント（カテゴリ変数）
for period in config["target_periods"]:
    direction_col = f'target_price_direction_{period}'
    if direction_col in feature_df.columns:
        value_counts = feature_df[direction_col].value_counts()
        total = value_counts.sum()
        print(f"\n{direction_col} の頻度分布:")
        for value, count in value_counts.items():
            percentage = (count / total) * 100
            print(f"  {value}: {count} ({percentage:.2f}%)")

# 特徴量間の相関行列（数値のみ表示）
if 'target_price_direction_1' in feature_df.columns:
    print("\n特徴量と目標変数の相関（絶対値が0.05以上のもののみ）:")
    corrs = feature_df.corr()['target_price_direction_1'].drop('target_price_direction_1')
    significant_corrs = corrs[abs(corrs) >= 0.05].sort_values(ascending=False)
    for feat, corr in significant_corrs.items():
        print(f"  {feat}: {corr:.6f}")

# データサンプル数値表示（最初の5行の主要特徴量のみ）
print("\n最初の5行のデータサンプル（主要特徴量のみ）:")
sampled_features = ['close', 'volume', 'rsi', 'macd', 'target_price_change_pct_1', 'target_price_direction_1']
if all(feat in feature_df.columns for feat in sampled_features):
    sample_df = feature_df[sampled_features].head(5)
    for i, row in sample_df.iterrows():
        print(f"\n行 {i}:")
        for col, val in row.items():
            print(f"  {col}: {val:.6f}" if isinstance(val, float) else f"  {col}: {val}")

2025-04-26 13:10:41,783 - feature_engineering - INFO - データを data\raw\btcusd_5m_data.csv から読み込みます
2025-04-26 13:10:41,939 - feature_engineering - INFO - 169358 行のデータを読み込みました
2025-04-26 13:10:41,941 - feature_engineering - INFO - 価格変動率の特徴量を追加しています
2025-04-26 13:10:41,997 - feature_engineering - INFO - 出来高変動率の特徴量を追加しています
2025-04-26 13:10:42,031 - feature_engineering - INFO - 移動平均線と乖離率の特徴量を追加しています
2025-04-26 13:10:42,067 - feature_engineering - INFO - RSIの特徴量を追加しています
2025-04-26 13:10:42,083 - feature_engineering - INFO - 高値/安値からの距離の特徴量を追加しています
2025-04-26 13:10:42,127 - feature_engineering - INFO - ボリンジャーバンドの特徴量を追加しています
2025-04-26 13:10:42,139 - feature_engineering - INFO - MACDの特徴量を追加しています
2025-04-26 13:10:42,149 - feature_engineering - INFO - ストキャスティクスの特徴量を追加しています
2025-04-26 13:10:42,170 - feature_engineering - INFO - 予測対象の目標変数を追加しています
2025-04-26 13:10:42,282 - feature_engineering - INFO - 特徴量を生成しました。カラム数: 114, 行数: 169156
2025-04-26 13:10:54,466 - feature_engineering - INFO - 特徴量を data\pr


=== 特徴量生成結果 ===
状態: success
行数: 169156
列数: 114
期間: 2023-01-01T15:00:00 から 2025-04-26T03:55:00
欠損値: 0

特徴量グループごとの数:
  price_change: 20
  volume_change: 17
  moving_averages: 20
  rsi: 5
  high_low_distance: 8
  bollinger_bands: 8
  macd: 4
  stochastic: 5
  target: 9

目標変数の分布:
  target_price_direction_1: {0: 74691, 1: 47381, -1: 47084}
  target_price_direction_2: {1: 56511, 0: 56450, -1: 56195}
  target_price_direction_3: {1: 61016, -1: 60514, 0: 47626}

主要特徴量の基本統計量:

close:
  count: 169156.000000
  mean: 53435.799885
  std: 25348.342357
  min: 16560.500000
  25%: 28502.950000
  50%: 54271.750000
  75%: 68760.700000
  max: 109059.400000

volume:
  count: 169156.000000
  mean: 388.673700
  std: 759.950594
  min: 1.612000
  25%: 104.689000
  50%: 197.543000
  75%: 396.478750
  max: 43836.312000

price_change_pct_1:
  count: 169156.000000
  mean: 0.000012
  std: 0.001888
  min: -0.090764
  25%: -0.000583
  50%: 0.000000
  75%: 0.000591
  max: 0.107624

volume_change_pct_1:
  count: 169156

# モデルトレーニングと評価のJupyter Notebook


In [ ]:
# モデルトレーニングと評価のJupyter Notebook

import sys
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# model_builderモジュールをインポート
sys.path.append(os.path.abspath(os.getcwd()))
from model_builder import train_models, evaluate_models, format_model_report

# モデルのトレーニング設定
training_config = {
    "input_dir": "data/processed",
    "input_filename": "btcusd_5m_features.csv",
    "output_dir": "models",
    "feature_groups": {
        "price": True,            # 価格関連特徴量
        "volume": True,           # 出来高関連特徴量
        "technical": True,        # テクニカル指標関連特徴量
    },
    "target_periods": [1, 2, 3],  # 予測対象期間 (1=5分後, 2=10分後, 3=15分後)
    "cv_splits": 5,               # 時系列交差検証の分割数
    "test_size": 0.2,             # テストデータの割合
    "regression": {               # 回帰モデル（価格変動率予測）設定
        "model_params": {
            "objective": "regression",
            "metric": "mae",
            "boosting_type": "gbdt",
            "num_leaves": 31,
            "learning_rate": 0.05,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.8,
            "bagging_freq": 5,
            "verbose": -1
        },
        "fit_params": {
            "num_boost_round": 1000,
            "early_stopping_rounds": 50,
            "verbose_eval": 100
        }
    },
    "classification": {           # 分類モデル（価格変動方向予測）設定
        "model_params": {
            "objective": "multiclass",
            "num_class": 3,       # 上昇/横ばい/下落の3クラス
            "metric": "multi_logloss",
            "boosting_type": "gbdt",
            "num_leaves": 31,
            "learning_rate": 0.03,  # 学習率を少し下げる
            "feature_fraction": 0.8,
            "bagging_fraction": 0.7,
            "bagging_freq": 5,
            "min_data_in_leaf": 50, # 過学習防止のために増加
            "max_depth": 8,        # 木の深さを制限
            "verbose": -1
        },
        "fit_params": {
            "num_boost_round": 2000, # ラウンド数を増加
            "early_stopping_rounds": 100,
            "verbose_eval": 100
        }
    }
}

# モデルのトレーニングを実行
print("モデルのトレーニングを開始します...")
training_report = train_models(training_config)

# トレーニング結果を表示
print("\n=== トレーニング結果 ===")

# 回帰モデルの結果
print("\n## 回帰モデル（価格変動率予測）")
for period_key, result in training_report["regression"].items():
    period = period_key.split("_")[1]
    print(f"\n### {period}期先予測")
    print(f"MAE: {result['mae']:.6f}")

    print("\n上位特徴量:")
    for feature, importance in result["top_features"].items():
        print(f"  {feature}: {importance:.6f}")

# 分類モデルの結果
print("\n## 分類モデル（価格変動方向予測）")
for period_key, result in training_report["classification"].items():
    period = period_key.split("_")[1]
    print(f"\n### {period}期先予測")
    print(f"正解率: {result['accuracy']:.4f}")

    print("\nクラスごとの精度:")
    for class_label, value in result["class_accuracy"].items():
        print(f"  {class_label}: {value:.4f}")

    print("\n上位特徴量:")
    for feature, importance in result["top_features"].items():
        print(f"  {feature}: {importance:.6f}")

# モデルの評価設定
evaluation_config = {
    "input_dir": "data/processed",
    "input_filename": "btcusd_5m_features.csv",
    "model_dir": "models",
    "output_dir": "evaluation",
    "target_periods": [1, 2, 3],  # 予測対象期間 (1=5分後, 2=10分後, 3=15分後)
    "test_size": 0.2               # テストデータの割合
}

# モデルの評価を実行
print("\nモデルの評価を開始します...")
evaluation_report = evaluate_models(evaluation_config)

# 評価結果を表示
print("\n=== 評価結果 ===")
eval_report_str = format_model_report(evaluation_report)
print(eval_report_str)

2025-04-26 13:10:57,813 - model_trainer - INFO - データを data\processed\btcusd_5m_features.csv から読み込みます


モデルのトレーニングを開始します...


2025-04-26 13:10:59,879 - model_trainer - INFO - 169156 行のデータを読み込みました
2025-04-26 13:10:59,880 - model_trainer - INFO - 選択された特徴量: 105個
2025-04-26 13:10:59,902 - model_trainer - INFO - 特徴量: 105個, 目標変数: 6個
2025-04-26 13:10:59,926 - model_trainer - INFO - トレーニングデータ: 135325行, テストデータ: 33831行
2025-04-26 13:10:59,927 - model_trainer - INFO - 1期先の価格変動率予測モデルをトレーニングします
2025-04-26 13:11:00,393 - model_trainer - INFO - 1期先の価格変動率予測モデル - MAE: 0.001097
2025-04-26 13:11:00,395 - model_trainer - INFO - モデルを models\regression_model_period_1.joblib に保存しました
2025-04-26 13:11:00,396 - model_trainer - INFO - 2期先の価格変動率予測モデルをトレーニングします


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	train's l1: 0.000903422	valid's l1: 0.00109693


2025-04-26 13:11:00,909 - model_trainer - INFO - 2期先の価格変動率予測モデル - MAE: 0.001585
2025-04-26 13:11:00,912 - model_trainer - INFO - モデルを models\regression_model_period_2.joblib に保存しました
2025-04-26 13:11:00,912 - model_trainer - INFO - 3期先の価格変動率予測モデルをトレーニングします


...

Early stopping, best iteration is:
[75]	train's multi_logloss: 1.01224	valid's multi_logloss: 1.05512
Training until validation scores don't improve for 50 rounds


2025-04-26 13:11:07,222 - model_trainer - INFO - 3期先の価格変動方向予測モデル - 正解率: 0.4096
2025-04-26 13:11:07,233 - model_trainer - INFO - モデルを models\classification_model_period_3.joblib に保存しました
2025-04-26 13:11:07,270 - model_evaluator - INFO - 回帰モデル（1期先）を読み込みました
2025-04-26 13:11:07,277 - model_evaluator - INFO - 回帰モデル（2期先）を読み込みました
2025-04-26 13:11:07,284 - model_evaluator - INFO - 回帰モデル（3期先）を読み込みました
2025-04-26 13:11:07,295 - model_evaluator - INFO - 分類モデル（1期先）を読み込みました
2025-04-26 13:11:07,304 - model_evaluator - INFO - 分類モデル（2期先）を読み込みました
2025-04-26 13:11:07,312 - model_evaluator - INFO - 分類モデル（3期先）を読み込みました
2025-04-26 13:11:07,313 - model_evaluator - INFO - データを data\processed\btcusd_5m_features.csv から読み込みます


[100]	train's multi_logloss: 1.0095	valid's multi_logloss: 1.04853
Early stopping, best iteration is:
[50]	train's multi_logloss: 1.02575	valid's multi_logloss: 1.04656

=== トレーニング結果 ===

## 回帰モデル（価格変動率予測）

### 1期先予測
MAE: 0.001097

上位特徴量:
  price_change_pct_6: 0.001085
  price_change_pct_10: 0.000921
  price_change_pct_3: 0.000686
  macd_signal: 0.000684
  price_change_pct_2: 0.000511
  lower_shadow: 0.000467
  volume_change_pct_9: 0.000411
  volume_sma_5: 0.000372
  macd: 0.000363
  volume_ratio_5: 0.000338

### 2期先予測
MAE: 0.001585

上位特徴量:
  volume_sma_20: 0.001476
  turnover: 0.001204
  macd: 0.001198
  sma_diff_pct_200: 0.001091
  volume_change_pct_3: 0.001054
  stoch_k: 0.000961
  dist_from_high_50: 0.000863
  volume_sma_5: 0.000819
  dist_from_high_20: 0.000805
  price_change_pct_4: 0.000680

### 3期先予測
MAE: 0.001948

上位特徴量:
  volume_change_pct_10: 0.003139
  dist_from_low_5: 0.001856
  volume_sma_10: 0.001690
  candle_size: 0.001354
  price_change_pct_4: 0.001141
  macd: 0.001067


2025-04-26 13:11:09,407 - model_evaluator - INFO - 169156 行のデータを読み込みました
2025-04-26 13:11:09,418 - model_evaluator - INFO - テストデータ: 33831行, 特徴量: 105個
2025-04-26 13:11:09,428 - model_evaluator - INFO - 回帰モデル（1期先）評価 - MAE: 0.001097
2025-04-26 13:11:09,439 - model_evaluator - INFO - 回帰モデル（2期先）評価 - MAE: 0.001599
2025-04-26 13:11:09,449 - model_evaluator - INFO - 回帰モデル（3期先）評価 - MAE: 0.001952
2025-04-26 13:11:09,494 - model_evaluator - INFO - 分類モデル（1期先）評価 - 正解率: 0.2653
2025-04-26 13:11:09,533 - model_evaluator - INFO - 分類モデル（2期先）評価 - 正解率: 0.3229
2025-04-26 13:11:09,573 - model_evaluator - INFO - 分類モデル（3期先）評価 - 正解率: 0.3344
2025-04-26 13:11:09,575 - model_evaluator - INFO - 評価レポートを evaluation\model_evaluation_report.json に保存しました



=== 評価結果 ===
=== モデル評価レポート ===

## 回帰モデル（価格変動率予測）

### 1期先予測
MAE: 0.001097

### 2期先予測
MAE: 0.001599

### 3期先予測
MAE: 0.001952

## 分類モデル（価格変動方向予測）

### 1期先予測
正解率: 0.2653

クラスごとの精度:
  下落: Precision=0.0000, Recall=0.0000, F1-score=0.0000, Support=10512.0
  横ばい: Precision=0.3000, Recall=0.0002, F1-score=0.0005, Support=12799.0
  上昇: Precision=0.3069, Recall=0.8529, F1-score=0.4514, Support=10520.0

混同行列:
実際\予測         下落(-1)    横ばい(0)     上昇(1)
下落(-1)             0         4      8658      1850
横ばい(0)             0         3     11604      1192
上昇(1)              0         3      8973      1544

### 2期先予測
正解率: 0.3229

クラスごとの精度:
  下落: Precision=0.0000, Recall=0.0000, F1-score=0.0000, Support=12322.0
  横ばい: Precision=0.2186, Recall=0.1210, F1-score=0.1558, Support=9075.0
  上昇: Precision=0.3670, Recall=0.7903, F1-score=0.5012, Support=12434.0

混同行列:
実際\予測         下落(-1)    横ばい(0)     上昇(1)
下落(-1)             0      2092      9372       858
横ばい(0)             0      1098      7575       402
上昇